<a href="https://colab.research.google.com/github/makhmudov-khondamir/Machine-Learning-Projects/blob/main/pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pnevmoniya:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F36499%2F3618301%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T113540Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1cd5f85f95c28cfb53db1915bca42c894bed7cfe88f3ab545c06d3c8a5db9a2a1ecd739d4da22af0936564b99c26812ab311f2ac1ee3d78b43d1e8cd4a3934d10057c80bf3f1be6ad4d0e94b6f5557d1eef248b102e08b4e184f903bd336ccac2af127ffe5e5ec53af43a1c24ec96dab12ccd546b99e963b375284c1c59270030a2a7cd694f96efa7dd1efb33179d19cf594935af938240b08ea47f312261c0c43482ffef759446a7e75d3b0ca86cfb9195f0b93034e03f7768462ca06905ad41746d81208dc9cbd9ab585f2471bd91d29d394d1e6e8700a0b256d609dca3b7c29078ca08945a4bf7c181522e16f829d58fc4b434b521f3262136b3ceeff3fbf,resnet:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5502775%2F9116506%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240806%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240806T113540Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D43ca8dc2336f0897fd093ba616c393e41d90fd9a3cdce4556825c1ac0e44b0809734c9d9a4266bd38c8ba49f6e83d02f48e078617d6b0dab5fc7f13b15cfb3b64a5a3de0a42edcf182c9d39b0adffb40795a6bc1ca0213929837227e6cd9e96f1b7c41be82d2dd15d96dfc5cd68a626bd6ff94063dc360e6cfd772ad36e7d7cce23ffbf0ee94faf73b3f0feea76dd8ce4b9d3c04fce8698b568e135a6993d984e192988736c938b349f12e6c4aef9cfdd14a7f563ae4b7906ef5584aa4e4962c91619648208552556b1098f352aeeb20422c3d62d8dd1b5964e00e2f8b53b0800f9f4f48d8814de2adc6fe8c8b2a2a90b442ee1c9e8977f291e7c97cb2561f2a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import fastai
import torch
from fastai.vision.all import *

print("fastai version: ", fastai.__version__)

In [ ]:
path = Path('/kaggle/input/pnevmoniya/train')
path.ls()

In [ ]:
p_datablock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter = RandomSplitter(seed=42),
    item_tfms=Resize(224)
)

df = p_datablock.dataloaders(path)
df.show_batch()

In [ ]:
from fastai.vision.all import *
import torch

# Define the path to your data
path = Path('/kaggle/input/pnevmoniya/train')

# Create the DataBlock
p_datablock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(seed=42),
    item_tfms=Resize(224)
)

# Create DataLoaders
dls = p_datablock.dataloaders(path)

# Define the path to the manually uploaded weights
weights_path = '/kaggle/input/resnet/resnet34-b627a593.pth'  # Update this path with your exact file path

# Define a custom function to load the model with specific weights
def custom_resnet34(weights_path):
    model = resnet34(pretrained=False)  # Do not use pretrained=True
    # Load weights from the file
    model.load_state_dict(torch.load(weights_path))
    return model

# Create a Learner with the custom model
learn = Learner(dls, custom_resnet34(weights_path), metrics=accuracy)

# Fine-tune the model
learn.fine_tune(3)


In [ ]:
test_path = Path('/kaggle/input/pnevmoniya/test')
test_path.ls()

# **Testing**

In [ ]:
sol = pd.read_csv('/kaggle/input/pnevmoniya/sample_solution.csv')
sol.head()

In [ ]:
def check(img):
    """
    Predict if the image is normal or pneumonia.
    """
    pred_class, _, _ = learn.predict(img)
    return 0 if pred_class == "NORMAL" else 1

# Function to get predictions for a set of images
def get_predicts(path, ids):
    results = []
    for id in ids:
        img = PILImage.create(path/str(id))
        res = check(img)
        results.append(res)
    return results

In [ ]:
sol["labels"] = get_predicts(test_path, sol["id"])
sol.to_csv("predictionn.csv", index=False)
sol.head()